In [1]:
%pip install pandas numpy seaborn matplotlib scikit-learn xgboost

     ---------------------------------------- 14.8/14.8 MB 4.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.1
    Uninstalling numpy-1.26.1:
      Successfully uninstalled numpy-1.26.1
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\MOHD SUFYAN\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~umpy.libs\\libopenblas64__v0.3.23-293-gc2f4bdbb-gcc_10_3_0-2bde3a66a51006b2b53eb373ff767a3f.dll'
Consider using the `--user` option or check the permissions.



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

import pickle as pkl

import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, f1_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler as Scaler
# import hyperopt

from models import decision_tree, k_nearest, logistic_regression, random_forest, support_vector_cls, xgboost

In [2]:
FILENAME = 'pyradiomics_extraction_segmentation_no_maskcorrect_with_FS'

RESULTS_PATH = f'Results without preprocessing/{FILENAME}.csv'
CLASS_LABELS = '../../Data/Patient class labels.csv'
DF_PATH = '../../Data/Without Demographic Features/pyradiomics_extraction_segmentation_no_maskcorrect.csv'
MODEL_PICKLING = f'Saved Models/{FILENAME}.pkl'

In [3]:
TRAIN_RATIO = 0.6
VAL_RATIO = 0.2
TEST_RATIO = 0.2

SEED = 2454259

FEATURE_SCALE = True
CRITERION = 'accuracy'

In [4]:
labels = pd.read_csv(CLASS_LABELS)
features = pd.read_csv(DF_PATH).drop(columns = 'sequence', errors='ignore')

total_features = pd.merge(features, labels, left_on = 'patient', right_on = 'Patient ID').drop(columns = ['Patient ID', 'patient'])

## Train test split

In [5]:
def train_test_val_splits(df, train_ratio = 0.8, val_ratio = 0.2, random_state = 2454259):
   
    val_ratio_adj = val_ratio / (1-train_ratio)

    train_df, val_df = train_test_split(df, train_size = train_ratio, random_state= random_state)
    val_df, test_df = train_test_split(val_df, train_size = val_ratio_adj, random_state= random_state)

    return train_df, val_df, test_df


train_df, val_df, test_df = train_test_val_splits(total_features, TRAIN_RATIO, VAL_RATIO, random_state = SEED)

In [7]:
train_x= train_df.drop(columns = ['ER', 'PR', 'HER2', 'Mol Subtype'])
train_y_er = train_df['ER'].to_numpy()
train_y_pr = train_df['PR'].to_numpy()
train_y_her = train_df['HER2'].to_numpy()
train_y_mol_subtype = train_df['Mol Subtype'].to_numpy()


val_x = val_df.drop(columns = ['ER', 'PR', 'HER2', 'Mol Subtype'])
val_y_er = val_df['ER'].to_numpy()
val_y_pr = val_df['PR'].to_numpy()
val_y_her = val_df['HER2'].to_numpy()
val_y_mol_subtype = val_df['Mol Subtype'].to_numpy()

test_x = test_df.drop(columns = ['ER', 'PR', 'HER2', 'Mol Subtype'])
test_y_er = test_df['ER'].to_numpy()
test_y_pr = test_df['PR'].to_numpy()
test_y_her = test_df['HER2'].to_numpy()
test_y_mol_subtype = test_df['Mol Subtype'].to_numpy()

## Training

In [8]:
if FEATURE_SCALE == True:
    scaler = Scaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    val_x = scaler.transform(val_x)
    test_x = scaler.transform(test_x)
    print('Features scaled')

Features scaled


## Fitting models

In [9]:
classifications_classes = {
    'ER' : (train_x, train_y_er, val_x, val_y_er, test_x, test_y_er),
    'PR' : (train_x, train_y_pr, val_x, val_y_pr, test_x, test_y_pr),
    'HER2': (train_x, train_y_her, val_x, val_y_her, test_x, test_y_her),
    'Mol Subtype': (train_x, train_y_mol_subtype, val_x, val_y_mol_subtype, test_x, test_y_mol_subtype)
}

In [10]:
def get_metrics(model, x, truey, split):
    pred_y = model.predict(x)
    acc = accuracy_score(truey, pred_y)
    prec = precision_score(truey, pred_y, average = 'weighted')
    rec = recall_score(truey, pred_y, average = 'weighted')
    f1 = f1_score(truey, pred_y, average = 'weighted')

    return {f'{split}_acc': acc, f'{split}_prec': prec, f'{split}_rec': rec, f'{split}_f1': f1}


In [11]:
def get_NB_model(tx, ty, vx, vy, testx, testy, classification = None):
    nb = GaussianNB()
    nb.fit(tx, ty)
    
    train_metrics = get_metrics(nb, tx, ty, 'train')
    val_metrics = get_metrics(nb, vx, vy, 'val')
    test_metrics = get_metrics(nb, testx, testy, 'test')

    combined_metrics = {**train_metrics, **val_metrics, **test_metrics}

    result = pd.DataFrame(combined_metrics, index = [0])
    result['Algorithm'] = 'Naive Bayes'
    if classification is not None:
        result['classification'] = classification

    return result, nb

def get_DT_model(tx, ty, vx, vy, testx, testy, criterion = 'accuracy', classification = None):
    '''Get Decision Tree model with metrics'''
    model = decision_tree.get_best_hyperparameters(tx, ty, vx, vy, criterion = criterion)

    train_metrics = get_metrics(model, tx, ty, 'train')
    val_metrics = get_metrics(model, vx, vy, 'val')
    test_metrics = get_metrics(model, testx, testy, 'test')

    combined_metrics = {**train_metrics, **val_metrics, **test_metrics}

    result = pd.DataFrame(combined_metrics, index = [0])
    result['Algorithm'] = 'Decision Tree'
    if classification is not None:
        result['classification'] = classification

    return result, model

def get_KNN_model(tx, ty, vx, vy, testx, testy, criterion = 'accuracy', classification = None):
    '''Get KNN model with metrics'''
    model = k_nearest.get_best_hyperparameters(tx, ty, vx, vy, criterion = criterion)

    train_metrics = get_metrics(model, tx, ty, 'train')
    val_metrics = get_metrics(model, vx, vy, 'val')
    test_metrics = get_metrics(model, testx, testy, 'test')

    combined_metrics = {**train_metrics, **val_metrics, **test_metrics}

    result = pd.DataFrame(combined_metrics, index = [0])
    result['Algorithm'] = 'KNN'
    if classification is not None:
        result['classification'] = classification

    return result, model

def get_LR_model(tx, ty, vx, vy, testx, testy, criterion = 'accuracy', classification = None):
    '''Get Logistic Regression model with metrics'''
    model = logistic_regression.get_best_hyperparameters(tx, ty, vx, vy, criterion = criterion)

    train_metrics = get_metrics(model, tx, ty, 'train')
    val_metrics = get_metrics(model, vx, vy, 'val')
    test_metrics = get_metrics(model, testx, testy, 'test')

    combined_metrics = {**train_metrics, **val_metrics, **test_metrics}

    result = pd.DataFrame(combined_metrics, index = [0])
    result['Algorithm'] = 'Logistic Regression'
    if classification is not None:
        result['classification'] = classification

    return result, model

def get_RF_model(tx, ty, vx, vy, testx, testy, criterion = 'accuracy', classification = None):
    '''Get Random forest model with metrics'''
    model = random_forest.get_best_hyperparameters(tx, ty, vx, vy, criterion = criterion)

    train_metrics = get_metrics(model, tx, ty, 'train')
    val_metrics = get_metrics(model, vx, vy, 'val')
    test_metrics = get_metrics(model, testx, testy, 'test')

    combined_metrics = {**train_metrics, **val_metrics, **test_metrics}

    result = pd.DataFrame(combined_metrics, index = [0])
    result['Algorithm'] = 'Random Forest'
    if classification is not None:
        result['classification'] = classification

    return result, model

def get_SVM_model(tx, ty, vx, vy, testx, testy, criterion = 'accuracy', classification = None):
    '''Get SVM model with metrics'''
    model = support_vector_cls.get_best_hyperparameters(tx, ty, vx, vy, criterion = criterion)

    train_metrics = get_metrics(model, tx, ty, 'train')
    val_metrics = get_metrics(model, vx, vy, 'val')
    test_metrics = get_metrics(model, testx, testy, 'test')

    combined_metrics = {**train_metrics, **val_metrics, **test_metrics}

    result = pd.DataFrame(combined_metrics, index = [0])
    result['Algorithm'] = 'SVM'
    if classification is not None:
        result['classification'] = classification

    return result, model
    
def get_XGB_model(tx, ty, vx, vy, testx, testy, criterion = 'accuracy', classification = None):
    '''Get XGBoost model with metrics'''
    model = xgboost.get_best_hyperparameters(tx, ty, vx, vy, criterion = criterion)

    train_metrics = get_metrics(model, tx, ty, 'train')
    val_metrics = get_metrics(model, vx, vy, 'val')
    test_metrics = get_metrics(model, testx, testy, 'test')

    combined_metrics = {**train_metrics, **val_metrics, **train_metrics}

    result = pd.DataFrame(combined_metrics, index = [0])
    result['Algorithm'] = 'XGBoost'
    if classification is not None:
        result['classification'] = classification

    return result, model

In [12]:
classification_results = pd.DataFrame()
models = {}

for key, value in classifications_classes.items():

    tx, ty, vx, vy, testx, testy = value
    
    res_nb, nb = get_NB_model(tx, ty, vx, vy, testx, testy, classification = key)
    res_dt, dt = get_DT_model(tx, ty, vx, vy, testx, testy, criterion = CRITERION, classification = key)
    res_knn, knn = get_KNN_model(tx, ty, vx, vy, testx, testy, criterion = CRITERION, classification = key)
    res_lr, lr = get_LR_model(tx, ty, vx, vy, testx, testy, criterion = CRITERION, classification = key)
    res_rf, rf = get_RF_model(tx, ty, vx, vy, testx, testy, criterion = CRITERION, classification = key)
    res_svm, svm = get_SVM_model(tx, ty, vx, vy, testx, testy, criterion = CRITERION, classification = key)
    res_xgb, xgb = get_XGB_model(tx, ty, vx, vy, testx, testy, criterion = CRITERION, classification = key)

    models[key] = {'Naive Bayes' : nb,
                   'Decision Trees' : dt,
                   'K Nearest Neighbours': knn,
                   'Logistic Regression' : lr,
                   'Random Forest' : rf,
                   'SVM': svm,
                   'XGBoost': xgb}

    classification_results = pd.concat([classification_results,
                                        res_nb,
                                        res_dt,
                                        res_knn,
                                        res_lr,
                                        res_rf,
                                        res_svm,
                                        res_xgb])

c:\Users\MOHD SUFYAN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\MOHD SUFYAN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\MOHD SUFYAN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\MOHD SUFYAN\A

In [13]:
classification_results

,train_acc,train_prec,train_rec,train_f1,val_acc,val_prec,val_rec,val_f1,test_acc,test_prec,test_rec,test_f1,Algorithm,classification
0,0.273056,0.719190,0.273056,0.150630,0.255435,0.812842,0.255435,0.111228,0.275676,0.436086,0.275676,0.146107,Naive Bayes,ER
0,0.772152,0.753257,0.772152,0.757377,0.793478,0.776936,0.793478,0.765294,0.702703,0.673259,0.702703,0.681840,Decision Tree,ER
0,0.819168,0.808564,0.819168,0.803707,0.804348,0.790707,0.804348,0.791512,0.681081,0.651666,0.681081,0.662040,KNN,ER
0,0.764919,0.753688,0.764919,0.692894,0.766304,0.774022,0.766304,0.687481,0.713514,0.617937,0.713514,0.621021,Logistic Regression,ER
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.718919,0.665501,0.718919,0.658903,Random Forest,ER
0,0.817360,0.842990,0.817360,0.777997,0.842391,0.869760,0.842391,0.813609,0.702703,0.634835,0.702703,0.642221,SVM,ER
0,0.896926,0.904492,0.896926,0.888337,0.891304,0.905063,0.891304,0.879880,NaN,NaN,NaN,NaN,XGBoost,ER
0,0.365280,0.665492,0.365280,0.218279,0.364130,0.770670,0.364130,0.200930,0.345946,0.444919,0.345946,0.204585,Naive Bayes,PR
0,0.679928,0.785623,0.679928,0.577329,0.673913,0.783830,0.673913,0.571093,0.637838,0.554317,0.637838,0.542407,Decision Tree,PR
0,0.773960,0.769008,0.773960,0.768824,0.760870,0.755637,0.760870,0.753380,0.578378,0.555732,0.578378,0.564018,KNN,PR


In [13]:
classification_results
cols = classification_results.columns.tolist()

cols = cols[-2:] + cols[:-2]
classification_results = classification_results[cols]

classification_results.to_csv(RESULTS_PATH, index = False)


with open(MODEL_PICKLING, 'wb') as file:
    pkl.dump(models, file)

OSError: Cannot save file into a non-existent directory: 'Results without preprocessing'